In [1]:
##Import necessary modules

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import tkinter as tk
import math
from matplotlib import pyplot as plt

In [3]:
##Import Data and build numpy arrays to manipulate

dataset = pd.read_csv('PressureSensorData.csv')
currentData = dataset
transPressure = np.array(currentData['pt_psia'])#.reshape(-1,1)
anaPressure = np.array(currentData['pa_psia'])
voltage = np.array(currentData['pd_volts'])

In [4]:
##Create function to get estimated Analogue Pressure for input voltage

def getAnaP(event):
    volts = float(userVoltIn.get())
    model = smf.ols('pa_psia ~ pd_volts', data=currentData)
    model = model.fit()
    result = model.params[0] + volts * model.params[1]
    anaRes.configure(text = "Your Estimated Analogue Readout is: " + str(result))

In [5]:
##Create function to get estimated True Pressure for input voltage

def getTrueP(event):
    volts = float(userVoltIn.get())
    model = smf.ols('pt_psia ~ pd_volts', data=currentData)
    model = model.fit()
    result = model.params[0] + volts * model.params[1]
    trueRes.configure(text = "Your Estimated True Pressure is: " + str(result))
    
    #Establish variables for calculating confidence interval
    sampSize = len(transPressure)                        #Sample size
    sampStd = np.std(transPressure)                      #Standard deviation (σ)
    
    #Find Margin of error
    margin = zValue*(sampStd/math.sqrt(sampSize))
    
    #Calculate our interval
    lowEnd = result-margin
    highEnd = result+margin
    
    confRes.configure(text = "With a Confidence Interval of (" + str(lowEnd) + "," + str(highEnd) + ")")

In [6]:
##Create function to find confidence interval

def findConf():
    
    #Select a z-value based on the selected radio-button
    confChoice = int(var.get())
    if(confChoice == 1):
        z = 1.645                      #The z value associated with 90% Confidence Interval
    elif(confChoice == 2):
        z = 1.960                      #The z value associated with 95% Confidence Interval
    elif(confChoice == 3):
        z = 2.576                      #The z value associated with 90% Confidence Interval
    global zValue
    zValue = z
    result = z

In [7]:
#### User Interface


#Initialize GUI
gui = tk.Tk()
gui.geometry("1000x500")

#Ask for user input
inputLabel = tk.Label(gui, text="Input Voltage:").pack()
userVoltIn = tk.Entry(gui)
userVoltIn.pack()

#Create radio buttons to select confidence interval
confidenceInput = tk.Label(gui,text="Select your desired Confidence Level")
confidenceInput.pack()
var = tk.StringVar(gui, "1")
values = {"90%" : "1","95%" : "2","99%" : "3"}
for (text, value) in values.items():
    tk.Radiobutton(gui, text = text, variable = var, value = value,command=findConf).pack()

#Bind return key to our functions
userVoltIn.bind("<Return>", getAnaP)
userVoltIn.bind("<Return>", getTrueP, add="+")

#Create a button to run our functions
bigOlButton = tk.Button(gui, text="Calculate",command=(lambda e=userVoltIn:[getAnaP(e),getTrueP(e)])).pack()

#Display our results
anaRes = tk.Label(gui)
trueRes = tk.Label(gui)
confRes = tk.Label(gui)
anaRes.pack()
trueRes.pack()
confRes.pack()



gui.mainloop()